In [ ]:
import numpy as np
import scipy.stats
from collections import Counter
import math 

def calculate_p_value_from_t_score(t_score, df):
    p_value = scipy.stats.t.sf((abs(t_score)), df) * 2
    return min(float(p_value), 0.99999)

confidence_level = 0.95

subjects_X_raters_matrix = np.array([
    [1, 2, "",3, 4],
    [2, 2, 3, 2,""],
    [1, 1, 1, 2, 2],
    [4, 4, 3, 4, 2],
    [3, 3, 4, "",3],
    [2, "",1, 1,""],
    [3, 3, 3, 4,""],
    [4, 3, 4, 3, 4],
    ["",3,"", "",""]
])

def Gwet_AC_Family (subjects_X_raters_matrix, weights_method = 'unweighted', confidence_level = 0.95):

    # Preperation of Values and Weights and Matrix
    Sample_Size = subjects_X_raters_matrix.shape[0]
    Number_Of_Raters = subjects_X_raters_matrix.shape[1]
    Strings_Vector = sorted(set(filter(None, subjects_X_raters_matrix.flatten())))
    Values_vector = np.array(sorted(set(map(float, filter(None, subjects_X_raters_matrix.flatten())))))

    Number_Of_Values = len(Strings_Vector)
    Max_Value = max(Values_vector)
    Min_Value = min(Values_vector)
    
    # Setting The Weights Method
    if weights_method == 'unweighted':
        Weights_Matrix = np.eye(Number_Of_Values)
    elif weights_method == 'linear':
        Weights_Matrix = 1 - (np.subtract.outer(Values_vector, Values_vector)**2) / (Max_Value - Min_Value)**2
    elif weights_method == 'quadratic':
        Weights_Matrix = 1 - abs(np.subtract.outer(Values_vector, Values_vector)) / abs(Max_Value - Min_Value)
    elif weights_method == 'ordinal':
        Weights_Matrix_Ordinal = (np.maximum.outer(Values_vector, Values_vector) - np.minimum.outer(Values_vector, Values_vector) + 1) * ( np.maximum.outer(Values_vector, Values_vector) - np.minimum.outer(Values_vector, Values_vector)) / 2 
        Weights_Matrix = 1 - Weights_Matrix_Ordinal/np.max(Weights_Matrix_Ordinal)
    elif weights_method == 'bipolar':
        Weights_Matrix_Bipolar1 = np.not_equal.outer(list(range(1, Number_Of_Values + 1)), list(range(1, Number_Of_Values + 1)))
        Weights_Matrix_Bipolar2 = np.subtract.outer(Values_vector, Values_vector)**2
        Weights_Matrix_Bipolar3 = np.add.outer(Values_vector, Values_vector)
        Weights_Matrix_Bipolar4 = np.where(Weights_Matrix_Bipolar1, (Weights_Matrix_Bipolar2) / (((Weights_Matrix_Bipolar3 - 2 *  Min_Value) * (2 * Max_Value - Weights_Matrix_Bipolar3))),0)
        Weights_Matrix = 1 - Weights_Matrix_Bipolar4/np.max(Weights_Matrix_Bipolar4)
    elif weights_method == 'circular':
        Weights_Matrix_Circular1 = np.sin(np.pi * (np.subtract.outer(Values_vector, Values_vector) / (Max_Value - Min_Value + 1)))**2
        Weights_Matrix = 1 - Weights_Matrix_Circular1 / np.max(Weights_Matrix_Circular1)
    elif weights_method == 'radical':
        Weights_Matrix = 1 - ( np.sqrt(abs(np.subtract.outer(Values_vector,Values_vector))) /  np.sqrt(abs(Max_Value-Min_Value)))
    elif weights_method == 'ratio':
        Weights_Matrix = 1 - (  ((np.subtract.outer(Values_vector,Values_vector)) / (np.add.outer(Values_vector,Values_vector)))**2) / (((Max_Value-Min_Value)/(Max_Value+Min_Value))**2)

    # Building an Agreement Matrix in a subjects_X_values format (Subject rows and values in coloumns)
    subjects_X_values_matrix = np.zeros((len(subjects_X_raters_matrix), len(Strings_Vector)), dtype=int)
    subjects_X_values_matrix = np.array([list(Counter(filter(None, row))[value] for value in Strings_Vector) for i, row in enumerate(subjects_X_raters_matrix)])[:Sample_Size]
    subjects_X_values_matrix_Weighted = np.transpose(np.dot(Weights_Matrix.T, np.transpose(subjects_X_values_matrix)))
 
    # Calculation of Gwet AC measure
    Raters_Counts_per_Sub = np.dot(subjects_X_values_matrix, np.ones(Number_Of_Values))
    Raters_Counts_per_Sub_Matrix = np.tile(Raters_Counts_per_Sub, (Number_Of_Values, 1)).T
    Q_Vector = np.sum(subjects_X_values_matrix * (subjects_X_values_matrix_Weighted-1), axis=1)
    Over_1_Rater = (Raters_Counts_per_Sub >= 2)
    Observed_Proportion_Agreement = np.sum(Q_Vector[Over_1_Rater] / (Raters_Counts_per_Sub[Over_1_Rater] * (Raters_Counts_per_Sub[Over_1_Rater] - 1))) / np.sum(Over_1_Rater)
    Expected_Agreement_Vector = np.sum(1/Sample_Size * (subjects_X_values_matrix/ Raters_Counts_per_Sub_Matrix), axis = 0)
    Expected_Agreement_Matrix = np.tile(Expected_Agreement_Vector, (Sample_Size, 1))
    Expected_Proportion_Agreement = np.sum(Weights_Matrix) * np.sum(Expected_Agreement_Vector * (1 - Expected_Agreement_Vector)) / (Number_Of_Values * (Number_Of_Values - 1))
    Gwet_AC = (Observed_Proportion_Agreement - Expected_Proportion_Agreement) / (1 - Expected_Proportion_Agreement)

    # Calculation of The Variance
    Var_Proportion_Observed= Q_Vector/((np.array(Raters_Counts_per_Sub) * (np.array(Raters_Counts_per_Sub)-1)) - ((np.array(Raters_Counts_per_Sub) * (np.array(Raters_Counts_per_Sub)-1)) == 0))
    Var_Proportion_Expected = np.sum((np.sum(Weights_Matrix)/(Number_Of_Values * (Number_Of_Values - 1))) * (subjects_X_values_matrix * (1- Expected_Agreement_Matrix)) / Raters_Counts_per_Sub_Matrix , axis = 1)
    Var_AC_Vector1 = (Sample_Size/np.sum(Over_1_Rater)) * (Var_Proportion_Observed-(Expected_Proportion_Agreement * (Over_1_Rater))) / (1-Expected_Proportion_Agreement)
    Varaince_Vector = Var_AC_Vector1 - 2 * (1 - Gwet_AC) * (Var_Proportion_Expected - Expected_Proportion_Agreement) / (1 - Expected_Proportion_Agreement)
    Standart_Error_Gwet = np.sqrt((1 / (Sample_Size * (Sample_Size - 1))) * np.sum((Varaince_Vector - Gwet_AC) ** 2))

    # Significance
    Statistic = Gwet_AC / Standart_Error_Gwet
    p_value = calculate_p_value_from_t_score(Statistic, Sample_Size-1)

    # Confidence Intervals
    tcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, Sample_Size-1)
    Lower_CI_Gwet = Gwet_AC - Standart_Error_Gwet*tcrit
    Upper_CI_Gwet = Gwet_AC + Standart_Error_Gwet*tcrit

    results = {}

    results["Gwet AC"]= Gwet_AC
    results["Statistic"]= Statistic
    results["p_value"] = p_value
    results["Standart Error Gwet's AC"] = Standart_Error_Gwet
    results["Confidence Intervals Gwet's AC"] = f"({round(Lower_CI_Gwet, 4)}, {round(Upper_CI_Gwet, 4)})"
   

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return results

Gwet_AC_Family(subjects_X_raters_matrix, weights_method="ordinal", confidence_level= 0.95)






